# Initiliation

In [1]:
!pip install transformers datasets --quiet

     |████████████████████████████████| 3.3 MB 5.3 MB/s 
     |████████████████████████████████| 298 kB 36.8 MB/s 
     |████████████████████████████████| 596 kB 54.5 MB/s 
     |████████████████████████████████| 3.3 MB 68.4 MB/s 
     |████████████████████████████████| 895 kB 55.1 MB/s 
     |████████████████████████████████| 61 kB 630 kB/s 
     |████████████████████████████████| 132 kB 62.4 MB/s 
     |████████████████████████████████| 243 kB 53.4 MB/s 
     |████████████████████████████████| 1.1 MB 53.2 MB/s 
     |████████████████████████████████| 271 kB 74.3 MB/s 
     |████████████████████████████████| 192 kB 63.1 MB/s 
     |████████████████████████████████| 160 kB 56.0 MB/s 


In [2]:
from transformers import TrainingArguments
from transformers import Trainer
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report, confusion_matrix
from datasets import Dataset
from datasets import load_metric

import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive

# Data Preparation

In [3]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
%cd 'drive/MyDrive/Masterarbeit/Colab Notebooks/OVERVIEW MBTI/Datasets/URL_Balanced_MASK'

/content/drive/.shortcut-targets-by-id/1aHXlqhpj1STohhfU4gn53D4whaLH__Jz/Masterarbeit/Colab Notebooks/OVERVIEW MBTI/Datasets/URL_Balanced_MASK


In [5]:
dfNS = pd.read_csv('MBTI_NS_URL_Balanced_MASK.csv', sep=",", error_bad_lines=False)
dfNS

,text,label
0,i like that you are kind as [MASK] i find that...,0
1,oh my you are right who really talks like tha...,0
2,yep yep yep especially the last one yep agree ...,0
3,things that are generalizable to the entire po...,0
4,work student hobbies studying gaming reading d...,0
...,...,...
1947,favorites one flew over the cuckoos nest1984 a...,1
1948,could i have my name changed to barkhouse than...,1
1949,i hate small talk just get to the point if its...,1
1950,when some days you talk with every people you ...,1


# Model Training

In [6]:
modeltype = "roberta-base"

In [7]:
train, test = train_test_split(dfNS, test_size=0.2, random_state=0, stratify=dfNS.label)

train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)

tokenizer = AutoTokenizer.from_pretrained(modeltype)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train = train.map(tokenize_function, batched=True)
tokenized_test = test.map(tokenize_function, batched=True)

full_train_dataset = tokenized_train
full_eval_dataset = tokenized_test

model = AutoModelForSequenceClassification.from_pretrained(modeltype, num_labels=2)

training_args = TrainingArguments(
    "RoBERTa_NS_MASK", 
    evaluation_strategy="epoch",
    save_strategy = 'no',
    save_steps = 100000,
    save_total_limit = 1,
    metric_for_best_model="eval_f1")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    print(classification_report(labels, preds, labels=[0,1]))
    print(confusion_matrix(labels,preds))
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

# Hyperparameter Optimization

In [8]:
! pip install optuna --quiet

     |████████████████████████████████| 308 kB 5.2 MB/s 
     |████████████████████████████████| 80 kB 11.3 MB/s 
     |████████████████████████████████| 209 kB 65.3 MB/s 
     |████████████████████████████████| 75 kB 5.5 MB/s 
     |████████████████████████████████| 49 kB 8.3 MB/s 
     |████████████████████████████████| 149 kB 72.3 MB/s 
     |████████████████████████████████| 112 kB 75.7 MB/s 


In [9]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(modeltype, num_labels=2)

In [10]:
trainer = Trainer(
      model_init=model_init,
      args=training_args, 
      train_dataset=full_train_dataset, 
      eval_dataset=full_eval_dataset,
      compute_metrics=compute_metrics 
  )

loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.13.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file https://huggingface.co/roberta-base/r

In [11]:
import sklearn.metrics as metrics
import optuna
import sys
import logging

def objective (metrics):
  return metrics['eval_f1']

def hyperparameter_space(trial):

    return {
        "learning_rate": trial.suggest_float("learning_rate", 5e-8, 5e-1, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [2, 4, 8, 16]),
        "weight_decay": trial.suggest_float("weight_decay", 5e-12, 5e-1, log=True),
        "num_train_epochs": trial.suggest_float("num_train_epochs",1,8,log=True),
        #"adam_epsilon": trial.suggest_float("adam_epsilon", 1e-10, 1e-6, log=True),
        #"seed" : trial.suggest_float("seed",10,60,log=True)
        }

optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "RoBERTa_MASK_NS"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)

best_run = trainer.hyperparameter_search(hp_space=hyperparameter_space,compute_objective=objective, n_trials=50, direction="maximize",study_name=study_name, storage=storage_name )

study = optuna.create_study()

[I 2021-12-12 17:02:21,689] A new study created in RDB with name: RoBERTa_MASK_NS


A new study created in RDB with name: RoBERTa_MASK_NS


Trial:
loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.13.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file https://huggingface.co/roberta

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.699600,0.693003,0.511509,0.338409,0.255754,0.500000
2,0.694300,0.701743,0.511509,0.338409,0.255754,0.500000
3,0.702800,0.693610,0.488491,0.328179,0.244246,0.500000
4,0.697900,0.693073,0.511509,0.338409,0.255754,0.500000
5,0.696900,0.693119,0.511509,0.338409,0.255754,0.500000
6,0.696300,0.693242,0.488491,0.328179,0.244246,0.500000
7,0.692800,0.692922,0.511509,0.338409,0.255754,0.500000
7,0.692700,0.692914,0.511509,0.338409,0.255754,0.500000


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, text.
***** Running Evaluation *****
  Num examples = 391
  Batch size = 8


...

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       200
           1       0.49      1.00      0.66       191

    accuracy                           0.49       391
   macro avg       0.24      0.50      0.33       391
weighted avg       0.24      0.49      0.32       391

[[  0 200]
 [  0 191]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param

Trial 49 pruned. 


[I 2021-12-12 19:46:16,642] A new study created in memory with name: no-name-39219a64-861d-4682-a20e-1ba445b59167


A new study created in memory with name: no-name-39219a64-861d-4682-a20e-1ba445b59167


In [12]:
storage_name

'sqlite:///RoBERTa_MASK_NS.db'

In [13]:
study_name

'RoBERTa_MASK_NS'

In [14]:
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True, direction="maximize")
df = study.trials_dataframe(attrs=("number", "value", "params", "state"))

[I 2021-12-12 19:46:16,753] Using an existing study with name 'RoBERTa_MASK_NS' instead of creating a new one.


Using an existing study with name 'RoBERTa_MASK_NS' instead of creating a new one.


In [15]:
df

,number,value,params_learning_rate,params_num_train_epochs,params_per_device_train_batch_size,params_weight_decay,state
0,0,0.338409,1.069600e-05,7.722648,2,8.320151e-08,COMPLETE
1,1,0.338409,4.221777e-01,5.688071,4,4.132608e-02,COMPLETE
2,2,0.440629,2.440031e-06,2.744239,8,5.252612e-07,COMPLETE
3,3,0.634118,1.081392e-05,2.981916,16,3.509982e-09,COMPLETE
4,4,0.616366,2.107441e-06,2.571573,4,4.113694e-11,COMPLETE
5,5,0.328179,2.308410e-06,4.656619,16,4.119576e-11,PRUNED
6,6,0.328179,5.303481e-03,7.334966,8,1.188048e-01,PRUNED
7,7,0.338409,8.714368e-05,1.361011,2,2.285913e-04,PRUNED
8,8,0.625451,3.017222e-05,3.355907,16,3.480614e-03,COMPLETE
9,9,0.338409,3.571422e-04,1.173250,8,1.629541e-11,PRUNED


In [16]:
fig = optuna.visualization.plot_param_importances(study)
fig.show()

In [17]:
best_run

BestRun(run_id='13', objective=0.6366386554621848, hyperparameters={'learning_rate': 2.9136645854693335e-05, 'num_train_epochs': 3.9728281289117455, 'per_device_train_batch_size': 16, 'weight_decay': 1.0946416634758784e-08})

In [18]:
optuna.visualization.plot_intermediate_values(study)

In [19]:
optuna.visualization.plot_parallel_coordinate(study)

In [20]:
optuna.visualization.plot_optimization_history(study)

In [21]:
optuna.visualization.plot_contour(study)

In [22]:
optuna.visualization.plot_slice(study)

In [23]:
optuna.visualization.plot_edf(study)